In [ ]:
"""new line in ingredients
"""

In [1]:
import os
import shutil
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

In [2]:
url="https://www.food.com/recipe/chicken-tikka-masala-316620"
page=requests.get(url)
soup=BeautifulSoup(page.content,'html.parser')

In [84]:
def get_ingredients(soup):
    ingredients=soup.find_all('ul',{'class':"ingredient-list"})
    ing=""
    for ingredient in ingredients:
        for li in ingredient.find_all('li'):
            if li.find('h4'):
                #current_heading = li.find('h4').text.strip()  # update the current heading
                continue
            ingredient_text = ''
            for span in li.find_all('span', {'class': 'ingredient-text'}):
                #ingredient_text += span.text.strip()
                ingredient_text += span.get_text(separator=' ', strip=True) + ' '
            for quantity in li.find_all('span', {'class': 'ingredient-quantity'}):
                ingredient_text = quantity.text.strip() + " " + ingredient_text
            ing+=ingredient_text.strip()+"\n"
    return ing

'1⁄2 teaspoon ground cumin 1⁄2 teaspoon ground coriander 1⁄4 teaspoon cayenne pepper 1 teaspoon table salt 2 lbs boneless skinless chicken breasts 1 cup plain yogurt (see note) 2 tablespoons    vegetable oil 2 medium garlic cloves , minced 1 tablespoon   grated fresh ginger 3 tablespoons    vegetable oil 1 medium onion , diced fine  (about 1 1/4 cups) 2 medium garlic cloves , minced 2 teaspoons   grated fresh ginger 1 fresh serrano chili , ribs and seeds removed, flesh minced  (see note) 1 tablespoon tomato paste 1 tablespoon garam masala (see note) 1 (28   ounce) can crushed tomatoes 2 teaspoons sugar 1⁄2 teaspoon table salt 2⁄3 cup heavy cream 1⁄4 cup   chopped fresh cilantro leaves '

In [100]:
ingredients=soup.find_all('ul',{'class':"ingredient-list"})
ing=""
for ingredient in ingredients:
    for li in ingredient.find_all('li'):
        if li.find('h4'):
            #current_heading = li.find('h4').text.strip()  # update the current heading
            continue
        ingredient_text = ''
        for span in li.find_all('span', {'class': 'ingredient-text'}):
            #ingredient_text += span.text.strip()
            ingredient_text += span.get_text(separator=' ', strip=True) + ' '
        for quantity in li.find_all('span', {'class': 'ingredient-quantity'}):
            ingredient_text = quantity.text.strip() + " " + ingredient_text
        ing+=ingredient_text.strip() + "\r\\n"
i2=ing.replace('\n',"\r")
ing

'1⁄2 teaspoon ground cumin\r\\n1⁄2 teaspoon ground coriander\r\\n1⁄4 teaspoon cayenne pepper\r\\n1 teaspoon table salt\r\\n2 lbs boneless skinless chicken breasts\r\\n1 cup plain yogurt (see note)\r\\n2 tablespoons    vegetable oil\r\\n2 medium garlic cloves , minced\r\\n1 tablespoon   grated fresh ginger\r\\n3 tablespoons    vegetable oil\r\\n1 medium onion , diced fine  (about 1 1/4 cups)\r\\n2 medium garlic cloves , minced\r\\n2 teaspoons   grated fresh ginger\r\\n1 fresh serrano chili , ribs and seeds removed, flesh minced  (see note)\r\\n1 tablespoon tomato paste\r\\n1 tablespoon garam masala (see note)\r\\n1 (28   ounce) can crushed tomatoes\r\\n2 teaspoons sugar\r\\n1⁄2 teaspoon table salt\r\\n2⁄3 cup heavy cream\r\\n1⁄4 cup   chopped fresh cilantro leaves\r\\n'

In [46]:
def getcookingtime(soup):
    duration=""
    c_time=soup.find_all('div',{'class':"facts"})
    c_time
    for c in c_time:
        c1=c.find('div',{'class':"facts__item"})
        cooking_time=c1.find('dd',{'class':"facts__value"})
        duration+=cooking_time.text
    return duration    

'1hr 10mins'

In [102]:
layout=soup.find('div',{'class':"current-recipe"})
u_list=layout.find('ul',{'class':"direction-list"})
directions=""
for li in u_list:
    directions+=li.text
directions+=""    
directions
    

'\nThis dish is best when prepared with whole-milk yogurt, but low-fat yogurt can be substituted. For a spicier dish, do not remove the ribs and seeds from the chile. If you prefer, substitute 2 teaspoons ground coriander, 1/4 teaspoon ground cardamom, 1/4 teaspoon ground cinnamon, and 1/2 teaspoon ground black pepper for the garam masala. The sauce can be made ahead, refrigerated for up to 4 days in an airtight container, and gently reheated before adding the hot chicken. Serve with basmati rice.FOR THE CHICKEN: Combine cumin, coriander, cayenne, and salt in small bowl. Sprinkle both sides of chicken with spice mixture, pressing gently so mixture adheres. Place chicken on plate, cover with plastic wrap, and refrigerate for 30 to 60 minutes. In large bowl, whisk together yogurt, oil, garlic, and ginger; set aside.FOR THE SAUCE: Heat oil in large Dutch oven over medium heat until shimmering. Add onion and cook, stirring frequently, until light golden, 8 to 10 minutes. Add garlic, ginger

In [75]:
print(u_list)

<ul class="direction-list svelte-1dqq0pw">
<li class="direction svelte-1dqq0pw">This dish is best when prepared with whole-milk yogurt, but low-fat yogurt can be substituted. For a spicier dish, do not remove the ribs and seeds from the chile. If you prefer, substitute 2 teaspoons ground coriander, 1/4 teaspoon ground cardamom, 1/4 teaspoon ground cinnamon, and 1/2 teaspoon ground black pepper for the garam masala. The sauce can be made ahead, refrigerated for up to 4 days in an airtight container, and gently reheated before adding the hot chicken. Serve with basmati rice.</li><li class="direction svelte-1dqq0pw">FOR THE CHICKEN: Combine cumin, coriander, cayenne, and salt in small bowl. Sprinkle both sides of chicken with spice mixture, pressing gently so mixture adheres. Place chicken on plate, cover with plastic wrap, and refrigerate for 30 to 60 minutes. In large bowl, whisk together yogurt, oil, garlic, and ginger; set aside.</li><li class="direction svelte-1dqq0pw">FOR THE SAUCE:

In [69]:
nutrition_wrappers=soup.find_all('div',{'class':"recipe-nutrition"})
for wrapper in nutrition_wrappers:
    wrapper_section=wrapper.find('section',{'class':"recipe-nutrition__info"})
nutrition_wrappers

[]

In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

PATH="C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\venv\\chromedriver.exe"
chrome_service=webdriver.ChromeService(executable_path=PATH)
chrome_options=Options()
chrome_options.add_experimental_option("detach",True)

# set up the Selenium driver
driver = webdriver.Chrome(service=chrome_service,options=chrome_options)
url = 'https://www.food.com/recipe/chicken-tikka-masala-316620'  # replace with the URL of your website
driver.get(url)

# find the button and click it
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'link.facts__nutrition')))
button.click()

# wait for the modal to appear and extract its HTML
time.sleep(15)
modal = driver.find_elements(By.CLASS_NAME,'nutrition-modal')
modal_html = modal.get_attribute('innerHTML')

# extract the nutrition data from the modal HTML
modal_soup = BeautifulSoup(modal_html, 'html.parser')
nutrition = modal_soup.find('div', {'class': 'modal-content'}).text.strip()
print(nutrition)

# close the Selenium driver
driver.quit()


AttributeError: 'list' object has no attribute 'get_attribute'

In [63]:
modal

[]

In [56]:
calories = soup.find('p', {'class': 'recipe-nutrition__item', 'class': 'bold', 'text': 'Calories:'})
if calories:
    calories_value = calories.find_next_sibling('p').text.strip()
    print('Calories:', calories_value)

In [58]:
print(calories)

None
